<center>机器学习期中大作业<br>第二组 / 王锦宏、吴泽辉、陶少聪、王海东、叶泽林、陈耀顺<br>RBF神经网络</center>

In [1]:
#运行本notebook需要以下库，如本cell报错，请先安装库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import math
from sklearn.model_selection import KFold
from scipy.io import loadmat

助教老师您好！在您调试我们的notebook时，可以修改mat_path_test为用于测试的数据集，我们会在最下方的cell对这一调试数据进行验证

In [2]:
#目录列表
#用于训练集的mat文件
mat_path = 'grp_2_train.mat'

#助教老师：此处为用于外部测试的mat文件目录，请修改为您的测试文件目录
mat_path_test = 'sample_test.mat'

In [3]:
#数据预处理函数
def matimport(path): #返回训练值和标记值
    m = loadmat(path)
    return m['feature'],m['label']

#输出标签数据处理函数
def Output_Matrix(input_y):#对label值进行一些调整，把0变为[1,0,0],1变[0,1,0],2变[0,0,1]
    size=input_y.shape[0]           
    op_y=np.zeros((size,3))
    for i in range(size):
        if input_y[i]==0:
            op_y[i][0]=1
        elif input_y[i]==1:
            op_y[i][1]=1
        else:
            op_y[i][2]=1
    return op_y

#结果统计函数
    #正确率统计函数
def get_accurancy(fit_y,test_y):
    accurate=0
    test_y = Output_Matrix(np.array(test_y).reshape(np.shape(fit_y)[0],1))
    for i in range(np.shape(fit_y)[0]):    #比较相同的个数
        if(i>np.shape(test_y)[0]):
            return (accurate/np.shape(test_y)[0])
            break
        if fit_y[i][0]==1 and fit_y[i][0]==test_y[i][0]:
            accurate+=1
        elif fit_y[i][1]==1 and fit_y[i][1]==test_y[i][1]:
            accurate+=1
        elif fit_y[i][2]==1 and fit_y[i][2]==test_y[i][2]:
            accurate+=1
    return (accurate/np.shape(fit_y)[0])

In [4]:
#有关计算函数
def MED(point1,point2):         #计算两点之间的欧式距离
    return (np.sum((point1-point2)**2))**0.5
    
def Gaussian(point1,point2,variance):  #径向基函数、高斯函数
    return math.exp(-(MED(point1,point2))**2/(2*variance**2))

def get_variance(center_point_number,center_point):   #计算方差
    max_MED=0
    for i in range(center_point_number):
        for j in range(center_point_number):
            med_=MED(center_point[i],center_point[j])
            if med_>max_MED:
                max_MED=med_
    return max_MED/((2*center_point_number)**0.5)

def get_wight(ip_x,center_point,center_point_number,op_y,variance):          #计算权值矩阵
    weight=np.zeros((center_point_number,3))      #权重矩阵
    j_matrix=np.zeros((center_point_number,np.shape(ip_x)[0])) #激励矩阵
    for i in range(center_point_number):
        for j in range(np.shape(ip_x)[0]):
            j_matrix[i][j]=Gaussian(center_point[i],ip_x[j], variance)
    j_inverse=np.linalg.pinv(j_matrix)    #激励矩阵的伪逆
    weight=np.dot(j_inverse.T,op_y)
    return weight 

In [5]:
#神经元操作
def tcal_y(input_x,weight):        #由神经元的输出乘以权重计算最终输出
    size=input_x.shape[0]
    y=np.array((size,3))
    y=np.dot(input_x,weight)
    return y

def cal_hide_output_matrix(input_x,center,center_point_number,variance):   #计算所有神经元的输出
    size=input_x.shape[0]#神经元个数
    hide_output_matrix=np.zeros((size,center_point_number))    
    for i in range(size):
        for j in range(center_point_number):   
            hide_output_matrix[i][j]=Gaussian(input_x[i],center[j], variance)  #利用高斯函数进行映射
    return hide_output_matrix

In [6]:
#K-Means聚类操作
def calc_center_point(cluster):         #计算簇的中心点
    size_of_cluster = len(cluster)            #簇中的点的个数
    center_point=np.zeros((1,70))
    for i in range(size_of_cluster):
        center_point=center_point+cluster[i]         #所有点加起来再除以个数
    center_point = center_point*(1/size_of_cluster)
    return center_point
 
    
def point_no_equal(point1,point2):   #判断两个中心点是否重合
    size=point1.shape[0]
    for i in range(size):
        if point1[i]!=point2[i]:
            return True
    return False

def check_center_diff(center, new_center):    #检查迭代之后形成的所有中心点与上一次是否相同
    n = len(center)
    for i in range(n):
        if point_no_equal(new_center[i],center[i]):
            return False
    return True

def k_means(input_x,k):  #k—means聚类方法
    center=input_x[:k]     #先曲前k个点作为中心点
    size=input_x.shape[0]   #总的点数
    cluster=[]              
    max_n=0              #最大迭代次数
    while True:
        for i in range(k):
            cluster.append([])
        for i in range(size):    #遍历每个点找到属于哪个簇
            min=1e5
            min_index=-1
            for j in range(k): 
                med_=MED(input_x[i],center[j])
                if min>med_:
                    min=med_
                    min_index=j
            cluster[min_index].append(input_x[i])
        temp_center=np.zeros((k,input_x.shape[1]))   #新的中心点
        for i in range(k):
            temp_center[i]=calc_center_point(cluster[i])
        if check_center_diff(center, temp_center):   #检查新旧的各个中心点是否相同，若相同停止聚类
            break
        max_n+=1
        if max_n>100:
            break
        center=temp_center    #更新所有中心点
        cluster=[]
    return center

In [7]:
class RBF_Regression():
    def __init__(self,feature,label,center_point_number=720):
        self.sourcedata_feature = feature
        self.sourcedata_label = label
        self.total_data_number=self.sourcedata_feature.shape[0] #总的样本数据个数
        self.trained = False
        self.center_point_number=center_point_number
    
    def train(self):
        if(self.center_point_number>self.total_data_number): 
            self.center_point_number = self.total_data_number
        ip_x = self.sourcedata_feature #特征向量矩阵
        ip_y = (self.sourcedata_label).reshape(-1,1) #读入label矩阵并计算
        self.k_center = k_means(ip_x[:],self.center_point_number) #k-means聚类
        self.var = get_variance(self.center_point_number,self.k_center) #高斯函数的方差
        output_train = Output_Matrix(ip_y)
        self.weight = get_wight(ip_x[:],self.k_center,self.center_point_number,output_train[:],self.var) #计算权重
        self.trained = True
    
    def estimate(self,test_feature): #直接调用模型对测试数据进行拟合
        if not self.trained: RBF_Regression.train(self)
        hide_output_matrix=cal_hide_output_matrix(test_feature[:],self.k_center,self.center_point_number,self.var)
        y=tcal_y(hide_output_matrix,self.weight)
        for i in range(np.shape(y)[0]):   #将输出的三个值先进行比较最大的变为1，剩下的全为0，与output统一
            if y[i][0]>y[i][1] and y[i][0]>y[i][2]:
                y[i][0]=1
                y[i][1]=0
                y[i][2]=0
            elif y[i][1]>y[i][0] and y[i][1]>y[i][2]:
                y[i][0]=0
                y[i][1]=1
                y[i][2]=0
            else:
                y[i][0]=0
                y[i][1]=0
                y[i][2]=1
        return y

In [12]:
#十折部分
train_x,train_y = matimport(mat_path)
test_x,test_y = matimport(mat_path_test)
kfold_rule = KFold(n_splits=10,random_state=23,shuffle=True)#打乱分割
train_y.reshape(720,1)
tenf_trainx = []
tenf_trainy = []
tenf_testx = []
tenf_testy = []
for train_index, test_index in kfold_rule.split(train_x):
    tenf_trainx.append(train_x[train_index])
    tenf_testx.append(train_x[test_index])
    tenf_trainy.append((train_y[0])[train_index])
    tenf_testy.append((train_y[0])[test_index])

In [13]:
for i in range(1,11):
    tenfoldmodel = RBF_Regression(tenf_trainx[i-1],tenf_trainy[i-1].reshape(1,-1))
    tenfoldmodel.train()
    fitresult = tenfoldmodel.estimate(tenf_testx[i-1])
    print("十折运算组%d拟合准确率：%f"%(i,get_accurancy(fitresult,tenf_testy[i-1].reshape(1,-1))))

mainmodel = RBF_Regression(train_x,train_y)
mainmodel.train()
mainfitresult = mainmodel.estimate(train_x)
print("全数据训练模型，对自身分类准确率：%f"%get_accurancy(mainfitresult,train_y))

outerfitresult = mainmodel.estimate(test_x)
print("------------------------")
print("使用外部数据测试全模型，分类准确率：%f"%get_accurancy(outerfitresult,test_y))

十折运算组1拟合准确率：0.958333
十折运算组2拟合准确率：0.958333
十折运算组3拟合准确率：0.986111
十折运算组4拟合准确率：0.972222
十折运算组5拟合准确率：1.000000
十折运算组6拟合准确率：0.986111
十折运算组7拟合准确率：0.986111
十折运算组8拟合准确率：0.972222
十折运算组9拟合准确率：0.986111
十折运算组10拟合准确率：0.986111
全数据训练模型，对自身分类准确率：1.000000
------------------------
使用外部数据测试全模型，分类准确率：1.000000
